# <center>CSCE4143-Final-Project</center>
##### <center>San Francisco Crime Classification</center>
##### <center>Group4: David Hammons | Logan Milazzo | Jose Martinez | Steve Liang</center>

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer

In [3]:
print(123)

123
